# Task 1

In [1]:
from src.sql_helpers import query_df
import matplotlib.ticker as ticker
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Connection to SQL server successful!


/home/markus/Desktop/Data Scientist/3. SQL/2. Examinerande_Uppgift/src/sql_helpers.py:19: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  with engine.connect():


### Get all unique ID's for Category, Subcategory and Product

In [5]:
# Get all unique ProductCategory IDs
category_query = """
SELECT ProductCategoryID, Name
FROM Production.ProductCategory
"""

unique_ids = {}

category_df = query_df(category_query)
productCategoryIDs = category_df['ProductCategoryID'].unique()
productCategoryIDs.sort()

# Get all unique ProductSubcategory IDs
subcategory_query = """
SELECT ProductSubcategoryID, ProductCategoryID
FROM Production.ProductSubcategory
"""
subcategory_df = query_df(subcategory_query)
productSubcategoryIDs = subcategory_df['ProductSubcategoryID'].unique()
productSubcategoryIDs.sort()

# Get all unique Product ID's
product_query = """
SELECT ProductID, ProductSubcategoryID
FROM Production.Product
"""

temp = query_df(product_query)
# Remove all NaN values
product_df = temp.dropna()
productIDs = product_df['ProductID'].unique()
productIDs.sort()


unique_ids["ProductCategoryID"] = list(productCategoryIDs)
unique_ids["ProductSubcategoryID"] = list(productSubcategoryIDs)
unique_ids["ProductID"] = list(productIDs)

### Group all Products to Subcategory

In [24]:
# Create a new DataFrame that contains a list of all product ID's for each Subcategory
# https://stackoverflow.com/questions/22219004/how-to-group-dataframe-rows-into-list-in-pandas-groupby
categorized_products = product_df.groupby("ProductSubcategoryID")["ProductID"].apply(list).reset_index(name="ProductID")
categorized_products["ProductSubcategoryID"] = categorized_products["ProductSubcategoryID"].astype(int)
categorized_products.head()

,ProductSubcategoryID,ProductID
0,1,"[771, 772, 773, 774, 775, 776, 777, 778, 779, ..."
1,2,"[749, 750, 751, 752, 753, 754, 755, 756, 757, ..."
2,3,"[953, 954, 955, 956, 957, 958, 959, 960, 961, ..."
3,4,"[808, 809, 810, 811, 812, 813, 946, 947]"
4,5,"[994, 995, 996]"


### Group all Subcategories to Category

In [ ]:
# Create a new DataFrame that contains a list of all Subcategory ID's for each Category
categorized_subcategories = subcategory_df.groupby("ProductCategoryID")["ProductSubcategoryID"].apply(list).reset_index(name="SubcategoryID")
categorized_subcategories.head()

,ProductCategoryID,SubcategoryID
0,1,"[1, 2, 3]"
1,2,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,..."
2,3,"[18, 19, 20, 21, 22, 23, 24, 25]"
3,4,"[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]"
